In [1]:
import os


In [2]:
ZIPFILE="./hymenoptera_data .zip"

In [3]:
from zipfile import ZipFile

In [4]:
with ZipFile(ZIPFILE) as f:
    f.extractall()

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [8]:
data_dir="./hymenoptera_data"

In [11]:
# transform

data_transform={
    "train":transforms.Compose([transforms.RandomResizedCrop(224),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()
                    ]),
    "val":transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()])
}

In [13]:
image_dataset={
    "train":datasets.ImageFolder(os.path.join(data_dir,"train")),
    
    "val":datasets.ImageFolder(os.path.join(data_dir,"val"))
}

In [14]:
dataloader={
    "train":torch.utils.data.DataLoader(image_dataset["train"], batch_size=4, shuffle=True,num_workers=4),
    
    "val":torch.utils.data.DataLoader(image_dataset["val"], batch_size=4, shuffle=True,num_workers=4)
}

In [16]:
data_sizes={
    "train":len(image_dataset["train"]),
    "val":len(image_dataset["val"])
}
data_sizes

{'train': 244, 'val': 153}

In [17]:
class_names=image_dataset["train"].classes

In [18]:
class_names

['ants', 'bees']